In [11]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import sys
import warnings
import pandas as pd
import numpy as np
import cx_Oracle as O
import datetime as datetime
import logging, logging.handlers, smtplib
import traceback
import smtplib
import sqlalchemy as sa
import pyodbc
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from os.path import basename
from datetime import datetime, date, timedelta
import settings
import errhandler as eh
import re
import requests
import urllib
import itertools
from requests.exceptions import ConnectionError, HTTPError, Timeout, TooManyRedirects
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from requests.auth import HTTPBasicAuth
from bs4 import BeautifulSoup
import time as ti

global dfrefs, sheets
dfrefs = pd.read_csv('LegalEntities.csv')
display(dfrefs)

col = dfrefs.pop('LE_ID')
dfrefs.insert(1, 'Provider ID', col)
display(dfrefs)

#fp = f'{os.path.basename(os.getcwd())}'
#filepath = fr'{fp}'
filepath = fr'{os.getcwd()}'
folderA = r'Archive'
filepathA = os.path.join(filepath, folderA)
folderB = r'xlsx'
filepathB = os.path.join(filepathA, folderB)
folderC = r'csv'
filepathC = os.path.join(filepathA, folderC)

dfrefs = dfrefs.replace({'St. Alexius Health Dickinson (St. Joseph’s) ED':'CHI St. Alexius Dickinson ED','St Alexius Health Devil’s Lake (Mercy) ED':"CHI St. Alexius Devil's Lake ED",
            'St Alexius Health Williston (Mercy) ED':'CHI St. Alexius Williston ED','Valley City University Student Health':'Egeberg, Pat',
            'Essentia ED':'Essentia Fargo ED','Jamestown Regional Medical Center':'Jamestown Regional Medical Center ED','Urgent Medicine Associated/Kurt Kooyer':'Kooyer, Kurt',
            'Northland Community Health Center McClusky':'Northland Community Health Center- McClusk','Central Dakota Clinic/Charles Nyhu':'Nyhus, Charles','Sanford Walk-in Fargo/Greg Pfennig':'Pfennig, Greg',
            'Sanford Medical Center Bismarck':'Sanford Medical Center Bismarck ED','Sanford Medical Center Fargo':'Sanford Medical Center Fargo ED',
            'Standing Rock IHS':'Standing Rock IHS','Trinity ED':'Trinity Medical Center ED','Wishek Community Clinic':'Wishek Community Clinic'})

#display(df, dfrefs)

def check_files():
    global processlist
    processlist = []
    for file in os.listdir(filepath):
        print(f'{file}')
        if "ILI-ND-WEEK" in file and file.endswith((".xlsx")):
            print(f'\n{file}')
            processlist += [file]
    if processlist == []:
        print('No Excel files were present')
        processlist += ['None']
    else:
        pass
    return processlist

def move_processed():
    global processedlist
    processedlist = []
    for file in os.listdir(filepath):
        print(f'{file}')
        if file.startswith("DDRProcessed") and file.endswith(".csv"):
            print(f'\n{file}')
            processedlist += [file]
        elif file.startswith(("Archive","Analysis","Temp")):
            pass
        elif ~file.startswith("DDRProcessed"):
            print('incorrect file name is present.')
    if processedlist == []:
        print('No files were present or did not have the correct naming scheme.')
        processedlist += ['None']
    else:
        pass
    return processedlist

def fludata(doc):
    
    df = pd.read_excel(f'{doc}')
    df = pd.merge(dfrefs, df, on = 'Provider ID')
    df = df.loc[:,~df.columns.str.contains('ILIDen')]
    df = df.loc[:,~df.columns.str.contains('MMWR')]
    df = df.loc[:,~df.columns.str.contains('unk')]
    df = df.loc[:,~df.columns.str.contains('Provider ID')]
    
    for row, cell in df.iterrows():
        
        XPATH = "xpath"
        
        url = 'https://ndhealth.co1.qualtrics.com/jfe/form/SV_77mVo06pgG4fQOO'
        my_web_form = requests.get(url).content
        soup = BeautifulSoup(my_web_form, 'html.parser')
        
        path = fr'{os.getcwd()}' + '\chromedriver.exe'
        options = webdriver.ChromeOptions()
        #options = Options()
        options.add_argument("--start-maximized")
        options.add_experimental_option("detach", True)
        #driver = webdriver.Chrome(service=Service(ChromeDriverManager.install()),options=options)
        driver = webdriver.Chrome(path, chrome_options=options)
        driver.get(url)
        try:
            elem = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.ID, "QR~QID6")))
        except Exception as e:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(e, 'line: ', exc_tb.tb_lineno,'\n\n', traceback.format_exc())
         
        inputdate = datetime.today().date() + timedelta(days=3)
        inputdate = datetime.strftime(inputdate, '%#m/%#d/%Y')
        
        selectors2 = driver.find_element(By.XPATH, "//div[contains(@class, 'QuestionOuter BorderColor MC  QID3')]")
        
        select2 = Select(driver.find_element(By.XPATH, "//select[contains(@class, 'ChoiceStructure Selection QR-QID3 QWatchTimer')]"))
        
        
        select2.select_by_visible_text(inputdate)
        
        user = os.getlogin() + ': ' + os.environ['COMPUTERNAME']
        user = str(user)
        
        username_div = driver.find_element(By.ID,'QR~QID6')
        
        username_div.send_keys(user)
        print(username_div)
        
        selectors3 = driver.find_element(By.XPATH, "//div[contains(@class, 'QuestionOuter BorderColor TE  QID4')]")
        #print(selectors3, '\n')
        text = selectors3.get_attribute("innerHTML")

        print(cell[0],cell[1],cell[2],cell[3],cell[4],cell[5])
        
        for i in range(1, -1, -1):
            print(f"{i} seconds until next document is processed", end ="\r", flush=True)
            ti.sleep(1)
        
        XPATH = "xpath"

        selectors = driver.find_element(By.XPATH, "//div[contains(@class, 'QuestionBody')]")

        select = Select(driver.find_element(By.XPATH, "//select[contains(@class, 'ChoiceStructure Selection QR-QID2 QWatchTimer')]"))

        select.select_by_visible_text(cell[0])
    
        selectors4 = driver.find_element(By.XPATH, "//td[contains(@class, 'ControlContainer')]")
        print('\n', selectors4)
        
        username_div = driver.find_element(By.ID,'QR~QID4~1')
    
        username_div.send_keys(cell[1])
        
        username_div = driver.find_element(By.ID,'QR~QID4~2')
    
        username_div.send_keys(cell[2])
    
        username_div = driver.find_element(By.ID,'QR~QID4~3')
    
        username_div.send_keys(cell[3])
        
        username_div = driver.find_element(By.ID,'QR~QID4~4')
    
        username_div.send_keys(cell[4])
        
        username_div = driver.find_element(By.ID,'QR~QID4~5')
    
        username_div.send_keys(cell[5])
        
        username_div = driver.find_element(By.ID,'QR~QID4~6')
    
        username_div.send_keys(cell[6])
        
        button = driver.find_element(By.XPATH, "//input[contains(@class, 'NextButton Button')]")
        
        button.click()
        
    return df



check_files()

if 'None' not in processlist:
    print('\ndocuments available for processing\n')
    print(processlist)
    for doc in enumerate(processlist):
        try:
            gstart_time = datetime.now()
            fludata(doc[1])
            gend_time = datetime.now()
            print("")
            print('\nDuration: {}'.format(gend_time - gstart_time))
            logging.info('\nDuration: {}'.format(gend_time - gstart_time))
            sys.exit()
        except Exception as e:
            print("")
            print(e)
            logging.exception('\n')
            logging.exception("message")
            continue
        for i in range(1, -1, -1):
                print(f"{i} seconds until next document is processed", end = "\r")
                ti.sleep(1)
else:
    print('documents not available for processing')
    pass

move_processed()
if 'None' not in processedlist:
    for file in processedlist:
        shutil.move(f'{filepath}'+'/'+f'{processedlist[0]}', filepath + '/Archive/'+f'{processedlist[0]}')
        move_processed()
    print('Processing Complete')
else:
    pass


,LE_ID,LE_Name
0,38008,Central Dakota Clinic/Charles Nyhus
1,38013,Valley City University Student Health
2,38043,UND Center for Family Medicine
3,38047,Altru ED
4,38048,Essentia ED
5,38049,Sanford Medical Center Bismarck
6,38050,Sanford Medical Center Fargo
7,38055,St. Alexius Health Dickinson (St. Joseph’s) ED
8,38056,Trinity ED
9,38058,Urgent Medicine Associated/Kurt Kooyer


,LE_Name,Provider ID
0,Central Dakota Clinic/Charles Nyhus,38008
1,Valley City University Student Health,38013
2,UND Center for Family Medicine,38043
3,Altru ED,38047
4,Essentia ED,38048
5,Sanford Medical Center Bismarck,38049
6,Sanford Medical Center Fargo,38050
7,St. Alexius Health Dickinson (St. Joseph’s) ED,38055
8,Trinity ED,38056
9,Urgent Medicine Associated/Kurt Kooyer,38058


.ipynb_checkpoints
Archive
chromedriver.exe
chromedriver_win32.zip
config.json
errhandler.py
ILI-ND-WEEK03-PROCESSED_03Feb23_09.27.29.xlsx

ILI-ND-WEEK03-PROCESSED_03Feb23_09.27.29.xlsx
ILI-ND-WEEK04-PROCESSED_03Feb23_09.27.29.xlsx

ILI-ND-WEEK04-PROCESSED_03Feb23_09.27.29.xlsx
ILI-ND-WEEK05-PROCESSED_03Feb23_09.27.29.xlsx

ILI-ND-WEEK05-PROCESSED_03Feb23_09.27.29.xlsx
ILIFluDataProject-Expansion-With-Selenium-Copy1.ipynb
ILIFluDataProject-Expansion-With-Selenium-Copy2.ipynb
ILIFluDataProject-Expansion-With-Selenium.ipynb
ILIWebAutomation.log
LegalEntities.csv
settings.py
__pycache__

documents available for processing

['ILI-ND-WEEK03-PROCESSED_03Feb23_09.27.29.xlsx', 'ILI-ND-WEEK04-PROCESSED_03Feb23_09.27.29.xlsx', 'ILI-ND-WEEK05-PROCESSED_03Feb23_09.27.29.xlsx']


C:\Users\gmoye\AppData\Local\Temp\3\ipykernel_21164\3725831246.py:113: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path, chrome_options=options)
C:\Users\gmoye\AppData\Local\Temp\3\ipykernel_21164\3725831246.py:113: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(path, chrome_options=options)


<selenium.webdriver.remote.webelement.WebElement (session="6536f5ce9eb405078614f2292758dec9", element="95fb83c4-abe8-40fa-8878-138126864836")>
Altru ED 0 0 1 0 0
0 seconds until next document is processed
 <selenium.webdriver.remote.webelement.WebElement (session="6536f5ce9eb405078614f2292758dec9", element="94198f9e-5a9c-4a77-b27c-dfe8e1b8c2d9")>


SystemExit: 

C:\Users\gmoye\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
getData()
transformation()